In [1]:
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [2]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable

large_model = init_chat_model("claude-sonnet-4-5")
standard_model = init_chat_model("gpt-5-nano")

# 메시지 길이를 카운팅하여 길면 큰모델, 적으면 작은 모델 사용
@wrap_model_call
def state_based_model(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """Select model based on State conversation length."""
    # request.messages is a shortcut for request.state["messages"]
    message_count = len(request.messages)  

    if message_count > 10:
        # Long conversation - use model with larger context window
        model = large_model
    else:
        # Short conversation - use efficient model
        model = standard_model

    request = request.override(model=model)  

    return handler(request)

In [3]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    middleware=[state_based_model],
    system_prompt="당신은 친절한 사무실 인턴 역할 극을 수행하고 있습니다."
)

In [4]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="오늘 화분에 물 줬어?")
        ]}
)

print(response["messages"][-1].content)

네, 오늘 물 줬어! 흙 표면이 살짝 마르는 느낌이라 흙 속까지 고르게 젖도록 충분히 주었어. 다음 물 주는 날짜를 기록해둘까? 아니면 알람도 맞춰줄까?


In [5]:
print(response["messages"][-1].response_metadata["model_name"])

gpt-5-nano-2025-08-07


In [6]:
from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?"),
        AIMessage(content="Yes, I gave it a light watering this morning."),
        HumanMessage(content="Has it grown much this week?"),
        AIMessage(content="It's sprouted two new leaves since Monday."),
        HumanMessage(content="Are the leaves still turning yellow on the edges?"),
        AIMessage(content="A little, but it's looking healthier overall."),
        HumanMessage(content="Did you remember to rotate the pot toward the window?"),
        AIMessage(content="I rotated it a quarter turn so it gets more even light."),
        HumanMessage(content="How often should we be fertilizing this plant?"),
        AIMessage(content="About once every two weeks with a diluted liquid fertilizer."),
        HumanMessage(content="When should we expect to have to replace the pot?")
        ]}
)

print(response["messages"][-1].content)

Probably in a few months – I've noticed the roots starting to peek through the drainage holes, so it'll need a bigger pot soon once it gets more root-bound.


In [7]:
print(response["messages"][-1].response_metadata["model_name"])

claude-sonnet-4-5-20250929
